In [1]:
!pip install requests
!pip install pandas
!pip install numpy
!pip install seaborn 
!pip install zipfile
!pip install tqdm

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [2]:
import requests
import pandas as pd
import seaborn as sns 
import numpy as np
import zipfile as zp
from tqdm import tqdm

In [3]:
launch_df = pd.read_csv('launchprices.csv')
yearly_df = pd.read_csv('yearlyspending.csv')
fda_df = pd.read_csv('fda_data.csv')

/var/folders/2f/5cmld2hs3flc3nvtxyfqq4gc0000gn/T/ipykernel_31666/2646210036.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  fda_df = pd.read_csv('fda_data.csv')


For the features, I think to improve runtime of merging and just generally optimize the process, it would make sense to store a feature vector in a dict that maps an index to that row in the directory, then we can basically just join w/o having to have the feature column in the vector and then we can add it back in post-merge.

In [4]:
#building feature dict
ing_cols = [col for col in fda_df.columns if col.startswith("ingredient")]
product_ids = fda_df['PRODUCTID']
ingredients = fda_df[ing_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
ingredients = fda_df[ing_cols]
feature_dict = {
    prod_id: np.array(ingredients.iloc[idx].astype(int).values)
    for idx, prod_id in enumerate(product_ids)
}

In [5]:
zero_sum_features = {key: vec for key, vec in feature_dict.items() if np.sum(vec) == 0}

# Count the number of feature vectors that sum to zero
num_zero_sum = len(zero_sum_features)

In [6]:
num_zero_sum

87261

In [7]:
len(feature_dict)

118467

In [8]:
# step to remove the ingredietn cols
fda_clean_df = fda_df.drop(columns=ing_cols)

In [9]:
yearly_df

,ndc,package_size,year,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
0,2010102,2,2024,ammonium c,34.000,27,0.00,0.00,0.00
1,2021301,1,2024,humulin r,26008.212,4582,333729.73,233776.39,99953.34
2,2120001,1,2024,amyvid,16.000,15,19252.97,19252.97,0.00
3,2120050,50,2024,NaN,12.000,12,4004.17,4004.17,0.00
4,2143301,1,2024,trulicity,1452.000,705,651142.34,648276.41,2865.93
...,...,...,...,...,...,...,...,...,...
178212,99207027028,28,2020,zyclara,843.500,44,48762.93,48756.93,6.00
178213,99207046630,30,2020,solodyn 80,390.000,13,838.96,818.16,20.80
178214,99999000005,5,2020,NaN,35.000,31,692.79,692.79,0.00
178215,99999163999,99,2020,NaN,119.000,28,21194.48,21194.48,0.00


In [10]:
ad_launch_df = launch_df.copy()
ad_launch_df['year'] = ad_launch_df['year'].replace(2019, 2020)

In [11]:
launch_year_df = pd.merge(
    ad_launch_df,
    yearly_df,
    on=['ndc', 'year'],
    how='inner'
)

In [12]:
launch_year_df

,ndc_description,ndc,nadac_per_unit,effective_date,pricing_unit,pharmacy_type_indicator,otc,explanation_code,classification_for_rate_setting,corresponding_generic_drug_nadac_per_unit,corresponding_generic_drug_effective_date,as_of_date,year,package_size,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
0,TRULICITY 0.75 MG/0.5 ML PEN,2143380,364.34368,2019-12-18,ML,C/I,N,"1, 5",B,NaN,NaN,01/01/2020,2020,80,trulicity,875510.000,287681,2.390402e+08,2.356648e+08,3375376.81
1,TRULICITY 1.5 MG/0.5 ML PEN,2143480,364.63237,2019-12-18,ML,C/I,N,"1, 5",B,NaN,NaN,01/01/2020,2020,80,trulicity,1775622.500,411175,3.483800e+08,3.432557e+08,5124328.10
2,EMGALITY 120 MG/ML PEN,2143601,555.45952,2019-11-20,ML,C/I,N,"2, 5",B,NaN,NaN,01/01/2020,2020,1,emgality,438.000,404,2.625435e+05,2.588458e+05,3697.72
3,EMGALITY 120 MG/ML PEN,2143611,555.45952,2019-11-20,ML,C/I,N,"2, 5",B,NaN,NaN,01/01/2020,2020,11,emgality,146702.243,133787,8.508297e+07,8.418795e+07,895017.71
4,TALTZ 80 MG/ML AUTOINJECTOR,2144501,6646.05125,2024-05-22,ML,C/I,N,"1, 5",B,NaN,NaN,05/22/2024,2024,1,taltz,16.000,14,1.195048e+04,1.195048e+04,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32054,FOSAPREPITANT 150 MG VIAL,83634077610,38.50800,2024-08-21,EA,C/I,N,"1, 5",G,NaN,NaN,08/21/2024,2024,10,fosaprepit,17.000,17,3.357200e+02,3.357200e+02,0.00
32055,FLUPHENAZINE 2.5 MG TABLET,90096012201,2.30140,2022-04-13,EA,C/I,N,4,G,NaN,NaN,04/13/2022,2022,1,fluphenazi,2402.000,36,4.476000e+03,4.475000e+03,1.00
32056,FLUPHENAZINE 5 MG TABLET,90096012301,2.98385,2022-04-13,EA,C/I,N,1,G,NaN,NaN,04/13/2022,2022,1,fluphenazi,585.000,11,1.158570e+03,1.158570e+03,0.00
32057,FLUPHENAZINE 10 MG TABLET,90096012401,3.52974,2022-04-13,EA,C/I,N,1,G,NaN,NaN,04/13/2022,2022,1,fluphenazi,3350.000,56,7.713980e+03,7.484280e+03,229.70


In [13]:
launch_year_df.to_csv('data.csv')
fda_clean_df.to_csv('fda_clean.csv')

In [14]:
launch_year_df['ndc'] = launch_year_df['ndc'].astype(str)
fda_clean_df['PRODUCTNDC'] = fda_clean_df['PRODUCTNDC'].astype(str)

fda_clean_df['PRODUCTNDC_no_hyphens'] = fda_clean_df['PRODUCTNDC'].str.replace('-', '')
fda_clean_df['PRODUCTNDC_zeros'] = fda_clean_df['PRODUCTNDC'].str.replace('-', '0')

launch_year_df['matching_product_ndc'] = None

for _, row in tqdm(fda_clean_df.iterrows(), total=fda_clean_df.shape[0], desc="Matching NDC Codes"):
    product_ndc_no_hyphen = row['PRODUCTNDC_no_hyphens']
    product_ndc_zero = row['PRODUCTNDC_zeros']
    
    matching_indices = launch_year_df['ndc'].str.startswith(product_ndc_no_hyphen)
    
    if not matching_indices.any():
        matching_indices = launch_year_df['ndc'].str.startswith(product_ndc_zero)
    
    launch_year_df.loc[matching_indices, 'matching_product_ndc'] = row['PRODUCTNDC']

launch_year_df = launch_year_df.dropna(subset=['matching_product_ndc'])

merged_df = launch_year_df.merge(fda_clean_df, left_on='matching_product_ndc', right_on='PRODUCTNDC', how='inner')


Matching NDC Codes: 100%|██████████████| 118467/118467 [11:38<00:00, 169.48it/s]


In [15]:
merged_df.sort_values(by='ndc')
merged_df.drop(columns=['PRODUCTNDC_no_hyphens', 'PRODUCTNDC_zeros'])

def get_feature(uniq_id):
    return feature_dict[uniq_id]

merged_df['FeatureVec'] = merged_df['PRODUCTID'].map(get_feature)
merged_df.to_csv('merged_data.csv')